In [28]:
#import all python libraries for numpy array calculation, biopython, pandas dataframe,hierarchical clustering, generation dendrogram and for visualization.
import os
import random as rd
import warnings
import numpy as np
from Bio import SeqIO
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib as mpl
import matplotlib as mp
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import colormaps
from matplotlib.figure import Figure
from matplotlib.colors import Normalize
from dtaidistance.subsequence.dtw import subsequence_alignment
from dtaidistance import dtw
from dtaidistance import ed
from dtaidistance import dtw_visualisation as dtwvis
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import average, dendrogram, linkage
import datetime
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 200) 


###########################User Input########################################
DistMeasure='DTW' #Manhattan, DTW or Euclidean
window=20
offset=2
UseThreshold=False
TopMatchesNum=20
ThresholdLow= 1.55
ThresholdHigh=1.75 #1.85 works well for DTW with 25 window and 5 offset,3 works well for Euclidean
InputPath="C:/Users/realt/Downloads/12 Known Prions.fasta"
OutputDirectory="C:/Users/realt/Downloads/PrionTestRuns/"

#############################################################################

#The values in the NoteValue array is generated based on physico chemical characteristics - MW and hydrophobicity of amino acids.
NoteValue = [0.3064,15.6497,60.7668,52.1362,76.6645,97.0000,-15.8315,-6.8806,5.5470,-2.3381,-23.9905,40.4569,60.1592,27.1418,42.9375,-22.8297,-11.7423,-17.1576,-33.9918,7.0585]

#columns array contains amino acids as allowed characters.  
columns=['G','A','L','M','F','W','K','Q','E','S','P','V','I','C','Y','H','R','N','D','T']
strAllowedChars = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L","M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]

#Generate a dictionary of amino acids as allowed characters and corresponding note value obtained based on physicochemical properties.
myDict = dict(zip(columns,NoteValue))

#Initialize vectors to store obtained peak values - single peaks and three peaks.
ThreePeakOutVectorAll=[]
SinglePeakOutVectorAll=[]
ThreePeakOutVectorAllP=[]
SinglePeakOutVectorAllP=[]

#Read and parse test fasta file containing prototypical and prion sequences using biopython module.
records = list(SeqIO.parse(InputPath, "fasta"))

#nameList is an empty list initialized to contain fasta header name of each protein sequence.
nameList=[]

#waveformsdf is initialized as empty dataframe to store cost matrix of dta distance value between each protein sequence.
waveformsdf = pd.DataFrame()

PRDRec = pd.DataFrame()


In [29]:
#Functions#######################################################
#Get the n lowest distance records
def lowest(a, n): return np.partition(a, n-1)[:n]

def WindowingFunc(seq,window=window, offset=offset):
    Searcher=[]
    startstopslst=[]
    for i in range(len(seq)):
        starter=(i*offset)-offset
        ender=starter+window
        start_stops=str(starter)+'_'+str(ender)
        currentsearch=seq[starter:ender]
        if len(currentsearch)==window:
            Searcher.append(currentsearch)
            startstopslst.append(start_stops)
    return Searcher,startstopslst

#Calculate Manhattan Distance
def ManhattanDist(vec1,vec2):
    distsum=np.sum(np.abs(vec1,vec2))
    return distsum

# Get top 10 records by distance
def TopMatches(df,TopMatches=TopMatchesNum):
    return df.nsmallest(TopMatches, 'Distance')

In [30]:
#Iterate through each protein fasta sequence of test multi fasta file and parse each of them to get fasta header name and fasta sequence.
VectorList=[]
nameList=[]
for i in range(0,len(records)):
    #extract fasta header of each protein sequence

    ##changing the long description to short id.
    names = records[i].id
    #Append each fasta header name to the list - nameList.
    nameList.append(names)
#     print (names)
    
    #Get the protein sequence as string value into currentseq variable. 
    currentseq = records[i].seq
    
    #Count the length of the sequence and store into lengther variable. 
    lengther=len(currentseq)
    

    ##This block of code looks for characters that are not amino acids and if any are found replaces them with a 'G'
    #Convert all of the charaters to upper before converting it to a list (JD)
    currentseq=currentseq.upper()

    #Convert the current sequence to a list (JD)
    currentseq=list(currentseq)

    #Loop through all of the characters and if any are not allowed amino acids, replace them with a 'G'(AK)
    for j in range(lengther):
            strAllowedChars = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L","M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]

            if currentseq[j] not in strAllowedChars: 
                currentseq[j] = currentseq[j].replace(currentseq[j],'G')
    #Convert the list back to a string (JD)
    currentseq=''.join(currentseq)

    ##   
    
    ##This block of code creates a vector containing only the notevalues representing the amino acid sequence (JD).
    NoteVec=[]
    ender=currentseq[-1]
    currentseq="G"+currentseq+ender
    lengther=len(currentseq)
#     print (currentseq)

    for k in range(lengther):
        if k==lengther:
            NoteVec.append(myDict[currentseq[k-1]])
        else:
            NoteVec.append(myDict[currentseq[k]])

    ##   
    
    ##This block of code converts a vector of NoteValues to a three-peak vector (JD)
    #Get the length of the NoteVector.
    lengther2=(len(NoteVec))-1
    
    #Empty lists are initialized to store the notevalue corresponding to each amino acid.
    ThreePeakOutVector=[]
    SinglePeakOutVector=[]

    #This block of code calculate PreOutvalue,Outvalue,PostOutvalue as three peak values for each amino acid position in protein fasta sequence.
    for l in range(lengther2):
        
        # This block of code is to claculate peak values for the first amino acid of the sequence.
        if l==0:
            PreOutvalue=NoteVec[l]/2
            Outvalue=((NoteVec[l]*2)+((NoteVec[l]+NoteVec[l+1])/2))/2
            PostOutvalue=(NoteVec[l]+((NoteVec[l]+NoteVec[l+1])/2))/2
            
            #Append peak value into the list - ThreePeakOutVector. 
            ThreePeakOutVector.append(round(PreOutvalue,4))
            ThreePeakOutVector.append(round(Outvalue,4))
            ThreePeakOutVector.append(round(PostOutvalue,4))
            
        # This block of code is to claculate peak values for  amino acid of the sequence.
        elif i==lengther:
            PreOutvalue=(NoteVec[l]+((NoteVec[l]+NoteVec[l-1])/2))/2
            Outvalue=((NoteVec[l]*2)+((NoteVec[l]+NoteVec[l-1])/2))/2
            PostOutvalue=NoteVec[i]
            
            #Append peak value into the list - ThreePeakOutVector.
            ThreePeakOutVector.append(round(PreOutvalue,4))
            ThreePeakOutVector.append(round(Outvalue,4))
            ThreePeakOutVector.append(round(PostOutvalue,4))
        
        # This block of code is to claculate peak values for the rest of the amino acids of the sequence.
        else:
            PreOutvalue=(NoteVec[l]+((NoteVec[l]+NoteVec[l-1])/2))/2
            Outvalue=((NoteVec[l])+((NoteVec[l]+NoteVec[l+1])/2)+((NoteVec[l]+NoteVec[l-1])/2))/2  
            PostOutvalue=(NoteVec[l]+((NoteVec[l]+NoteVec[l+1])/2))/2
            
            #Append peak value into the list - ThreePeakOutVector.
            ThreePeakOutVector.append(round(PreOutvalue,4))
            ThreePeakOutVector.append(round(Outvalue,4))
            ThreePeakOutVector.append(round(PostOutvalue,4)) 
            
    #This extracts the single peak vector from the three peak vector (JD)        
    SinglePeakOutVector=ThreePeakOutVector[1::3]
    ThreePeakOutVectorAll.append(np.array(ThreePeakOutVector))
    SinglePeakOutVectorAll=np.array(SinglePeakOutVector)
    
    
    VectorList.append(SinglePeakOutVectorAll)
    
RawVecsDF=pd.DataFrame()
RawVecsDF['Name']=nameList
RawVecsDF['SPVEC']=VectorList

# display(RawVecsDF)

TempLengthslst=[]
for i in range(len(RawVecsDF)):
    TempLengthslst.append(len(RawVecsDF.iloc[i]['SPVEC']))
    
TempNamezlst=list(RawVecsDF.Name)

TempLengthsDict=dict(map(lambda i,j : (i,j) , TempNamezlst,TempLengthslst))


In [31]:

windowslst=[]
vecwindowslst=[]
for i in range(len(RawVecsDF)):
    windowFullReturn=WindowingFunc(RawVecsDF.iloc[i]['SPVEC'])
    windows=windowFullReturn[1]
    vecwindows=windowFullReturn[0]
    windowslst.append(windows)
    vecwindowslst.append(vecwindows)
RawVecsDF['Range']=windowslst
RawVecsDF['WindowVecs']=vecwindowslst



In [32]:
temporarydistlist=[]
tempnamelist=[]
temprangelist=[]
temporarydistlist=[]
querynamelist=[]
queryrangelist=[]
querydistlist=[]

for i in range(len(RawVecsDF)):
    tempvecnames=RawVecsDF.iloc[i]['Name']
    tempvecwindows=RawVecsDF.iloc[i]['WindowVecs']
#     print(len(tempvecwindows))
    tempvecrange=RawVecsDF.iloc[i]['Range']
#     print(len(tempvecrange))
    for j in range(len(RawVecsDF)):
        queryvecnames=RawVecsDF.iloc[j]['Name']
        queryvecwindows=RawVecsDF.iloc[j]['WindowVecs']
#         print(len(queryvecwindows))
        queryvecrange=RawVecsDF.iloc[j]['Range']
#         print(len(queryvecrange))
        
        
        for k in range(len(tempvecwindows)):
            tempmeasseqvec=tempvecwindows[k]
            tempmeasseqrange=tempvecrange[k]
            for l in range(len(queryvecwindows)):
                querymeasseqvec=queryvecwindows[l]
                querymeasseqrange=queryvecrange[l]
                if DistMeasure=='DTW':
                    distance=dtw.distance_fast(tempmeasseqvec,querymeasseqvec)
                    distance=round(distance/window,2)
                elif DistMeasure=='Euclidean':
                    distance=ed.distance_fast(tempmeasseqvec,querymeasseqvec)
                    distance=round(distance/window,2)
                elif DistMeasure=='Manhattan':
                    distance=ManhattanDist(tempmeasseqvec,querymeasseqvec)
                    distance=round(distance/window,2)
                
                
                tempnamelist.append(tempvecnames)
                temprangelist.append(tempmeasseqrange)
                
                querynamelist.append(queryvecnames)
                queryrangelist.append(querymeasseqrange)
                temporarydistlist.append(distance)

MatchedFragsDF=pd.DataFrame()
MatchedFragsDF['TempName']=tempnamelist
MatchedFragsDF['TempRange']=temprangelist
MatchedFragsDF['QueryName']=querynamelist
MatchedFragsDF['QueryRange']=queryrangelist
MatchedFragsDF['Distance']=temporarydistlist

display(MatchedFragsDF)

,TempName,TempRange,QueryName,QueryRange,Distance
0,SUP35_P05453,0_25,SUP35_P05453,0_25,0.00
1,SUP35_P05453,0_25,SUP35_P05453,5_30,3.56
2,SUP35_P05453,0_25,SUP35_P05453,10_35,3.44
3,SUP35_P05453,0_25,SUP35_P05453,15_40,2.55
4,SUP35_P05453,0_25,SUP35_P05453,20_45,3.41
...,...,...,...,...,...
1760924,PRNP_P04156,225_250,PRNP_P04156,205_230,8.51
1760925,PRNP_P04156,225_250,PRNP_P04156,210_235,4.44
1760926,PRNP_P04156,225_250,PRNP_P04156,215_240,9.53
1760927,PRNP_P04156,225_250,PRNP_P04156,220_245,5.09


In [33]:
if UseThreshold is True:
    ThreshFilteredDistDF = MatchedFragsDF[(MatchedFragsDF['Distance'] >= ThresholdLow) & (MatchedFragsDF['Distance'] <= ThresholdHigh)]
    ThreshFilteredDistDF=ThreshFilteredDistDF[ThreshFilteredDistDF['TempName']!=ThreshFilteredDistDF['QueryName']]
#     display(ThreshFilteredDistDF)
elif UseThreshold is False:
    ThreshFilteredDistDFpre=MatchedFragsDF[MatchedFragsDF['TempName']!=MatchedFragsDF['QueryName']]
    ThreshFilteredDistDF = ThreshFilteredDistDFpre.groupby('TempName', group_keys=False).apply(TopMatches)
#     display(ThreshFilteredDistDF)



In [34]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize

# Get a list of the unique templates in ThreshFilteredDistDF
UniqueTemps = np.unique(list(ThreshFilteredDistDF.TempName))
# print(UniqueTemps)

#Get a timestamp to use in the filenames
TimeStamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

#Make the output directory if it does not exist
OutDirectoryFinal=OutputDirectory+str(TimeStamp)+"/"
if not os.path.exists(OutDirectoryFinal):
    os.makedirs(OutDirectoryFinal)
    
#Get a runtype stamp to use in the filename
if UseThreshold is True:
    RunType=str(DistMeasure)+"_Threshold_"+str(ThresholdLow)+"_"+str(ThresholdHigh)
elif UseThreshold is False:
    RunType=str(DistMeasure)+"_Top_"+str(TopMatchesNum)

for temp in UniqueTemps:
    UniqueTempDF = ThreshFilteredDistDF[ThreshFilteredDistDF['TempName'] == temp]
    TempName = temp
    XMax = int(TempLengthsDict[temp])
    
    # Plot settings
    figsize_width = 6  # Fixed width
    barheight = 0.2  # Consistent height for bars
    figsize_height = (len(UniqueTempDF) + 1) * barheight  # Adjust height based on number of bars
    
    fig, ax = plt.subplots(figsize=(figsize_width, figsize_height))
    
    # Normalize distance for color mapping
    norm = Normalize(vmin=min(ThreshFilteredDistDF.Distance), vmax=max(ThreshFilteredDistDF.Distance))
    cmap = cm.get_cmap('plasma')
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    
    # Add the template bar at the top
    ax.barh(y=barheight * len(UniqueTempDF), width=XMax, height=barheight, color='Black')
    
    ticklabels = [TempName]
    ticklocations = [barheight * len(UniqueTempDF)]
    
    # Add the fragments
    for j in range(len(UniqueTempDF)):
        distance = UniqueTempDF.iloc[j]['Distance']
        start = int(UniqueTempDF.iloc[j]['TempRange'].split("_")[0])
        width = int(UniqueTempDF.iloc[j]['TempRange'].split("_")[1]) - start  # Actual width of fragment
        yloc = barheight * j#(len(UniqueTempDF) - j - 1)
        color = cmap(norm(distance))
        
        ax.barh(yloc, width, left=start, height=barheight, color=color)
        
        fullname = f"{UniqueTempDF.iloc[j]['QueryName']}_{UniqueTempDF.iloc[j]['QueryRange']}"
        ticklabels.append(fullname)
        ticklocations.append(yloc)
        
    ax.margins(x=0, y=0)
    ax.set_xlim(0, XMax)
    ax.set_yticks(ticklocations)
    ax.set_yticklabels(ticklabels, fontsize=6)
    
    # Remove padding
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0.3)  # Adjusted bottom to leave space for colorbar
        
    plt.savefig(OutDirectoryFinal+TempName+"_"+RunType+"_win_"+str(window)+"_off_"+str(offset)+".png", bbox_inches='tight', dpi=300)
    plt.close()

# Create a standalone color bar that you can add to an aggregate figure later in Publisher, Illustrator, etc.
fig, ax = plt.subplots(figsize=(6, .125))  #Size as needed
cbar_ax = fig.add_axes([0.3, 0.3, 0.4, 0.5])  #Position/size
fig.colorbar(sm, cax=cbar_ax, orientation="horizontal", label='Distance')
ax.set_axis_off()
plt.savefig(OutDirectoryFinal+"ColorBar"+"_"+RunType+"_win_"+str(window)+"_off_"+str(offset)+".png", bbox_inches='tight', dpi=300)
plt.close()

MatchedFragsDF.to_pickle(OutDirectoryFinal+"AllMatchesDF"+"_"+DistMeasure+"_win_"+str(window)+"_off_"+str(offset)+".pkl")
ThreshFilteredDistDF.to_pickle(OutDirectoryFinal+"PlottedResultsDF"+"_"+DistMeasure+"_win_"+str(window)+"_off_"+str(offset)+".pkl")
